<a href="https://colab.research.google.com/github/PremGorecki/MachineLearning/blob/main/07_lasy_losowe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
iris = load_iris()
X = iris.data[:, 2:] # Długość i szerokość płatka
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], 
    voting='hard')



voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
# Przyjżyjmy się dokładności  klasyfikatora

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 1.0
RandomForestClassifier 0.98
SVC 1.0
VotingClassifier 1.0


### BaggingClassifier

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500, #500 drzew
                            max_samples=100, 
                            bootstrap=True, #True->Agregacja(ze zwracaniem)
                                            #False->wklejanie
                            n_jobs=-1) # -1 -> wszystkie rdzenie procka
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(bag_clf.__class__.__name__, accuracy_score(y_test, y_pred))

BaggingClassifier 1.0


In [6]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators=500, #500 drzew
                            max_samples=100, 
                            bootstrap=False, #True->Agregacja(ze zwracaniem)
                                            #False->wklejanie
                            n_jobs=-1) # -1 -> wszystkie rdzenie procka
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(bag_clf.__class__.__name__, accuracy_score(y_test, y_pred))

BaggingClassifier 0.98


In [7]:
bag_clf.oob_score

False

In [8]:
# Za pomocą poniższego kodu wyuczymy losowy las składający
# się z 500 drzew (każde ograniczamy do 16 węzłów) 
# za pomocą wszystkich dostępnych rdzeni procesora

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [9]:
# Poniżej zaprezentowana klasa BaggingClassifier stanowi z grubsza odpowiednik
# poprzednio zdefiniowanego klasyfikatora RandomForestClassifier:

bag_clf = BaggingClassifier(
DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

In [10]:
# Moduł Scikit-Learn po treningu automatycznie oblicza wynik dla każdej cechy, a następnie skaluje
# rezultaty tak, że suma wszystkich istotności wynosi 1.

from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.09116444319111061
sepal width (cm) 0.02309508498687488
petal length (cm) 0.4568984157629136
petal width (cm) 0.4288420560591008
